# O algoritmo RSA

O algoritmo RSA envolve quatro passos: geração de chaves, distribuição dessas chaves, cifragem e decifragem.
O principio por trás do RSA são três número inteiros positivos muito grandes, chamados de e, d e n.

## Geração das chaves

Selecione dois número primos distintos, simulares em magnitude mas pouco diferente na quantidade de digitos para tornar a fatoração dificil.

No nosso exemplo utilizaremos números pequenos para reduzir a complexidade do tutorial, uma vez que estamos interessados na lógica.

In [5]:
p = 11
q = 29

Calcule o número n que é a multiplicação dos dois números primos. O número n será utilizado como módulo para a definição das chaves públicas e privadas

In [6]:
n = p * q

Calcule a função Phi. Como p e q são primos a função Phi de p é p-1 e a função Phi de qé q-1

In [11]:
phi_of_n = (p - 1) * (q - 1)

print(phi_of_n)

280


Vamos importar o função para retornar o maior dividor comum entre dois números. Essa função é utilizada na lógica de geração da chave privada

In [20]:
from math import gcd
import random

Abaixo temos a definição de uma função que gera a chave privada

In [24]:
def get_encryption_key(n, phi_of_n):
    lst = [i for i in range(1, n+1)]
    e_list = []
    for i in lst:
        if (1 < i) and (i < phi_of_n):
            gcd_value = gcd(i, n)
            gcd_phi = gcd(i, phi_of_n)
            if (gcd_value == 1) and (gcd_phi == 1):
                e_list.append(i)
    if len(e_list) == 1:
        return e_list[0]
    else:
        return e_list[random.randint(1, len(e_list)-1)]  

E essa outra função gera a chave pública

In [25]:
def get_decryption_key(e, phi_of_n):
    d_list = []
    for i in range(e * 25):
        if (e * i) % phi_of_n == 1:
            d_list.append(i)
    return d_list[random.randint(1, len(d_list) - 1)]

In [28]:
e = get_encryption_key(n, phi_of_n)
d = get_decryption_key(e, phi_of_n)

# Utilizada para evitar colisão
while d == e:
    d = get_decryption_key(e, phi_of_n)

chave_publica = [e, n]
chave_privada = [d, n]

print(chave_privada)
print(chave_publica)

[1787, 319]
[123, 319]


## Distribuição das chaves

Idealmente a chave privada não deve ser transmitida. Ela deve ser gera no próprio disposito onde será utilizada, como equipamentos do tipo HSM (Hardware Security Module) que não permitam sua extração. Caso seja necessário a transmissão da chave privada, deve ser utilizado um meio confiável.
Já a chave pública deve ser distribuida para que seja possível a critografia da mensagem pela outra parte. Nesse tutorial não vamos precisar distribuir a chave pública pois rodaremos todo o processo nesse notebook

## Cifragem da mensagem

Primeiro vamos definir uma função que retorna uma lista com os valores ASCII de cada caracter da mensagem:

In [29]:
import string
def text_to_digits(PT):
    pool = string.ascii_letters + string.punctuation + " "
    M = []
    for i in PT:
        M.append(pool.index(i))
    return M

E agora definimos a função que faz a cifragem da mensagem usando a chave pública:

In [31]:
def encrypt(M, chave_publica):
    return [(i ** chave_publica[0]) % chave_publica[1] for i in M]

O passo final é cifrarmos a mensagem com as funções definidas acima

In [32]:
mensagem = "Hello, world!"
mensagem_em_ascii = text_to_digits(mensagem)

mensagem_cifrada = encrypt(mensagem_em_ascii, chave_publica)
print(mensagem_cifrada)

[121, 295, 242, 242, 269, 303, 101, 209, 269, 128, 242, 247, 281]


## Decifragem da mensagem

De forma semelhante a cifragem da mensagem, vamos definir uma função auxilidar que retorna o caracter da mensagem a partir do seu valor ASCII

In [33]:
def digits_to_text(DT):
    pool = string.ascii_letters + string.punctuation + " "
    msg = ''
    for i in DT:
        msg += pool[i]
    return msg

E agora definimos a função que faz a decifragem da mensagem:

In [35]:
def decrypt(CT, private_key):
    return [((i ** chave_privada[0]) % chave_privada[1]) for i in CT]

O passo final é utilizarmos esass funções para decifrar a mensagem:

In [37]:
mensagem_decifrage_em_ascii = decrypt(mensagem_cifrada, chave_privada) 
mensagem_aberta = original_PT = digits_to_text(mensagem_decifrage_em_ascii)

print(mensagem_aberta)

Hello, world!


Dessa forma concluímos o exemplo do algorimo RSA, definindo as chaves pública e privada, cifrando e decifrando uma mensagem com essas chaves.

# O algoritmo do Shor

O algoritmo de Shor é um algoritmo capaz encontrar os fatores de números primos que compõem um número inteiro.

Esse é justamente o passo contrário ao RSA, que a partir de dois números primos compõem o número n que é sua base. Como vimos acima precisamos ter acesso a esse número n tanto para cifrar como para decifrar uma mensagem.
Portanto para decifrarmos uma mensagem cifrada no RSA precisamos de acesso ao número n, um computador quântico com capacidade de execução do Shor para esse número n e a mensagem cifrada.

Acima temos definido o número n da nossa cifragem. A partir dele vamos obter pelo algorithmo de Shor seus fatores, que são os números base para recontrução das chave privada e pública.

O algoritmo de Shor é explicado em diversos materiais sobre computação quântica. Na sessão referências desse tutorial você entrará alguns links com a explicação.

No restante desse tutorial vamos utilizar o Qiskit e o simulador de computador quântico da IBM para demonstrar o uso do algoritmo de Shor para obter os números p e q que definimos no início desse texto.

Primeiro vamos importar as funções necessárias:

In [2]:
import math
import numpy as np
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Shor
from qiskit import IBMQ

O próximo passo é carregarmos a o token da nossa conta no IBM Experience para conseguirmos rodar o simulador e não depender dos recursos da nossa máquina local.

Também vamos obter o objeto que representa o backend desse simulador:

In [3]:
IBMQ.load_account()

provider = IBMQ.get_provider(group='open')
backend = provider.get_backend('ibmq_qasm_simulator')

Para finalizar criaremos o circuito do algoritmo de Shor para o valore de n que precisamos através de funções do Qiskit e executaremos esse circuito paara obter os valores de n e p definidos no início desse tutorial:

In [ ]:
quantum_instance = QuantumInstance(backend, shots=30)
shor = Shor(quantum_instance=quantum_instance)
result = shor.factor(n)
print(f"The list of factors of {n} as computed by the Shor's algorithm is {result.factors[0]}.")

Dessa forma mostramos um exemplo prático de como o algoritmo de Shor tem a capacidade de recuperar os fatores iniciais do algoritmo RSA e a partir desses fatores recriar as chaves privada e pública e decifrar uma mensagem.

## Referências

Algoritmo de Shor: https://en.wikipedia.org/wiki/Shor%27s_algorithm

Algoritmo RSA: https://en.wikipedia.org/wiki/RSA_(cryptosystem)

Explicação do algoritmo de Shor: https://qiskit.org/textbook/ch-algorithms/shor.html

Documentação do Qiskit: https://qiskit.org/documentation/tutorials/algorithms/08_factorizers.html

Página inicial da IBM Quantum Experience: https://quantum-computing.ibm.com/

Configuração da conta da IBM Quantum Experience: https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq

